# Проект "Определение уровня английского для фильмов"

В данном проекте необходимо определить, какой уровень английского языка представлен в фильме. Уровень английского языка определяется в соответствии с уровнем Oxford CEFR. Определение уровня осуществляется на основе субтитров к фильму. В дополнение, можно добавлять фильмы и субтитры к ним из открытых источников, поскольку первоначально в дасете 241 фильм.

**План работы:**
1. [Предобработка данных](#data_preprocessing)
2. [Расширение датасета](#dataset_expansion)
3. [Выбор метрики](#metric)
4. [Создание модели](#model)
5. [Анализ результатов](#analysis)

<a name='data_preprocessing'></a>
## 1. Предобработка данных

### 1. 1. Общий файл с фильмами

In [294]:
# Импортирование необходимых библиотек
import numpy as np
import pandas as pd
import PyPDF2
import pysrt
import re

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/midle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/midle/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [167]:
# Импорт файла
df_movies = pd.read_excel('movies_labels.xlsx')

df_movies.drop('id', axis=1, inplace=True)
df_movies.columns = ['movie', 'level']

print('Размер:', df_movies.shape)
df_movies.head()

Размер: (499, 2)


,movie,level
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2/A2+


In [172]:
# Замена промежуточный уровень на нижний
df_movies.loc[df_movies['level'] == 'A2/A2+', 'level'] = 'A2'
df_movies.loc[df_movies['level'] == 'A2/A2+, B1', 'level'] = 'A2'
df_movies.loc[df_movies['level'] == 'B1, B2', 'level'] = 'B1'

df_movies['level_num'] = df_movies['level'].map({
    'A1': 0,
    'A2': 1,
    'B1': 2,
    'B2': 3,
    'C1': 4
})

df_movies.head(5)

,movie,level,level_num
0,10_Cloverfield_lane(2016),B1,2
1,10_things_I_hate_about_you(1999),B1,2
2,A_knights_tale(2001),B2,3
3,A_star_is_born(2018),B2,3
4,Aladdin(1992),A2,1


In [173]:
# Пропущенные значения
df_movies.isna().sum()

movie        0
level        0
level_num    0
dtype: int64

In [179]:
# Распределение количества фильмов по уровням 
df_movies['level'].value_counts()

A2    177
B2    129
B1     92
C1     58
A1     43
Name: level, dtype: int64

### 1. 2. Файлы с субтитрами

In [8]:
# Вычленение то, что не относится к словам
HTML = r'<.*?>'
TAG = r'{.*?}'
COMMENTS = r'[\(\[][A-Za-z ]+[\)\]]'
UPPER = r'[[A-Za-z ]+[\:\]]'
LETTERS = r'[^a-zA-Z\'.,!? ]'
SPACES = r'([ ])\1+'
DOTS = r'[\.]+'
SYMB = r"[^\w\d'\s]"

# Функция для загрузки и лемматизации субтитров
def saving_subs(film_loc):
    # Загрузка субтитров по кодированию
    film_subs = []
    encodings = ['', 'UTF-8-SIG', 'ISO-8859-1', 'utf-8', 'Windows-1252', 'ascii']
    encoding_number = 0

    # Проверка возможности правильности кодирования, иначе - использование другого
    while not film_subs:
        try:
            film_subs = pysrt.open(
                film_loc,
                encoding=encodings[encoding_number]
            )
        except UnicodeDecodeError:
            encoding_number += 1

    # Удаление лишнего и оставление только слов
    fiml_subs = film_subs[1:]
    text = re.sub(HTML, ' ', film_subs.text)
    text = re.sub(TAG, ' ', text)
    text = re.sub(COMMENTS, ' ', text)
    text = re.sub(UPPER, ' ', text)
    text = re.sub(LETTERS, ' ', text)
    text = re.sub(DOTS, r'.', text)
    text = re.sub(SPACES, r'\1', text)
    text = re.sub(SYMB, '', text)
    text = re.sub('www', '', text)
    text = text.lstrip()
    text = text.encode('ascii', 'ignore').decode()
    text = text.lower()

    # Лемматизация слов и добавление их в отдельный список
    film_words = []
    text_list = text.split()
    lemmatizer = WordNetLemmatizer()

    for i in range(len(text_list)):
        word = lemmatizer.lemmatize(text.split()[i])
        # Проверка на наличие слова в списке
        if word not in film_words:
            film_words.append(word)

    # Объединение слов в одну строчку
    film_words = " ".join(film_words)
    return film_words

In [185]:
# Получение субтитров для каждого фильма, воспользовавшись функцией для обработки субтитров
for index, row in df_movies.iterrows():
    print(index)
    try:
        subs = saving_subs(f"subtitles/{row['movie']}.srt")
        df_movies.loc[df_movies['movie'] == row['movie'], 'subs'] = subs
    except FileNotFoundError:
        pass

235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484


In [188]:
# Удаление фильмов, к которым не были найдены субтитры
df_movies = df_movies[(df_movies['subs'] != '') & (df_movies['subs'].notna())]

### 1. 3. Слова по уровню языка

Создадим список со всеми словами, а также отдельный словарь с ключами в виде уровней английского и значениями - индексами начала слов уровня - 1.

In [11]:
# Ссылки на слова (А1-C1)
a1_b2_link = '/Users/midle/Desktop/Coding/Data Science/Projects/English_level/Oxford_CEFR_level/The_Oxford_3000_by_CEFR_level.pdf'
b2_c1_link = '/Users/midle/Desktop/Coding/Data Science/Projects/English_level/Oxford_CEFR_level/The_Oxford_5000_by_CEFR_level.pdf'

In [12]:
# Функция для сохранения слов в список из PDF-файла 
def get_level_words(link):
    reader = PyPDF2.PdfReader(link)
    
    words = ''
    for n in range(len(reader.pages)):
        words += reader.pages[n].extract_text()
        
    words = words.split('\n')
    words = [words[n].split()[0] for n in range(len(words)) if n > 1]
    return words

In [13]:
# Сохранение слов по уровням в отдельные переменные
a1_b2_words = get_level_words(a1_b2_link)
b2_c1_words = get_level_words(b2_c1_link)

# Внесем некоторые корректировки
a1_b2_words[1] = 'a'
a1_b2_words.insert(2, 'an')
b2_c1_words.remove('3000')
b2_c1_words.remove('B2')

In [14]:
# Объединим слова
level_words = a1_b2_words
level_words.extend(b2_c1_words)
level_words[:10]

['A1',
 'a',
 'an',
 'about',
 'above',
 'across',
 'action',
 'activity',
 'actor',
 'actress']

In [15]:
# Создание словаря с ключами в виде уровня английского и значениями в виде индексов начала слов
# данного уровня минус единица
levels = ['A1', 'A2', 'B1', 'B2', 'C1']
levels_dict = {level: level_words.index(level) for level in levels}

print(levels_dict)

{'A1': 0, 'A2': 891, 'B1': 1752, 'B2': 2550, 'C1': 3958}


In [126]:
'''Не улучшает качество моделей'''

'''
# Определение количества слов по уровням для каждого фильма
for index, row in df_movies.iterrows():
    # Словарь с подсчетом слов по каждому уровню для отдельного фильма
    film_level_words = {level: [] for level in levels}
    
    # Добавление в словарь слов по их уровням
    for word in row['subs'].split():
        try:
            word_index = level_words.index(word.lower())
            if levels_dict['A1'] < word_index < levels_dict['A2']:
                if word not in film_level_words['A1']:
                    film_level_words['A1'].append(word)
            elif levels_dict['A2'] < word_index < levels_dict['B1']:
                if word not in film_level_words['A2']:
                    film_level_words['A2'].append(word)
            elif levels_dict['B1'] < word_index < levels_dict['B2']:
                if word not in film_level_words['B1']:
                    film_level_words['B1'].append(word)
            elif levels_dict['B2'] < word_index < levels_dict['C1']:
                if word not in film_level_words['B2']:
                    film_level_words['B2'].append(word)
            else:
                if word not in film_level_words['C1']:
                    film_level_words['C1'].append(word)
        except ValueError:
            pass
    
    # Подсчет количества слов по уровням
    levels_words_count = {level: len(film_level_words[level]) for level in levels}
    
    # Добавление количества слов по уровням в отдельные столбцы
    total_words = sum(levels_words_count.values())
    for level in levels:
        df_movies.loc[df_movies['movie'] == row['movie'], level] = levels_words_count[level] / total_words
'''

<a name='metric'></a>
## 2. Выбор метрики

Имеем дело с задачей мультиклассификации. В связи с этим необходимо использовать соответствующие метрики определения качества моделей. Назначение проекта - помочь учащимся, изучающих английский язык. В связи с этим лучше всего использовать F1-меру, поскольку она сочетает в себе и точность (precision), и полноту (recall). Однако, поскольку мы имеем дело с мультиклассификацией, то необходимо воспользоваться "Макро F1-мерой" (деомнстрирующей среднее значение F1-меры по классам) и "Микро F1-мерой" (глобальная средняя F1-меры, вычисляющая сумму TP, FN и FP).

Таким образом, для определения качества моделей будем использовать **f1_micro** и **f1_macro**.

<a name='model'></a>
## 3. Создание модели

### 3. 1. Разделение выборок

In [189]:
# Разделение датасета на тренировочную и тестовую выборки

X = df_movies['subs']
y = df_movies['level_num']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=df_movies['level_num']
)

In [190]:
# Функция для нахождения предсказаний
def get_y_pred(model):
    vectorizer = TfidfVectorizer(stop_words={'english'})
    scaler = StandardScaler()
    
#     preprocessing = ColumnTransformer([
#         ('vectorizer', vectorizer, 'subs'),
#         ('scaler', scaler, levels),
#     ])
    
#     pipe = Pipeline([
#         ('preprocessing', preprocessing),
#         ('model', model)
#     ])
    
    pipe = Pipeline([
        ('vectorizer', vectorizer),
        ('model', model)
    ])
    
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    return y_pred

In [307]:
def print_f1_score(model, pred):
    # Проверка DecisionTreeClassifier на тестовой выборке
    print(f"{model}:")
    print("\tf1_micro: {:.4f}".format(f1_score(y_test, pred, average='micro')))
    print("\tf1_macro: {:.4f}".format(f1_score(y_test, pred, average='macro')))

### 3. 2. DecisionTreeClassifier

In [308]:
model = DecisionTreeClassifier(random_state=42, )
y_pred = get_y_pred(model)
print_f1_score(model, y_pred)

DecisionTreeClassifier(random_state=42):
	f1_micro: 0.5859
	f1_macro: 0.5516


In [138]:
# Использование GridSearchCV для DecisionTreeClassifier

'''
model = DecisionTreeClassifier(random_state=42)
params = {
    'model__max_depth': range(2, 21, 2),
    'model__criterion': ['gini', 'entropy']
}

preprocessing = ColumnTransformer([
    ('vectorizer', vectorizer, 'subs'),
    ('scaler', scaler, levels),
])

pipe = Pipeline([
    ('preprocessing', preprocessing),
    ('model', model)
])

grid = GridSearchCV(pipe, params, scoring=scoring, refit='f1_micro')
grid.fit(X_train, y_train)

metrics_cols = [f'mean_test_{x}' for x in scoring]
final_metrics = pd.DataFrame(grid.cv_results_)[metrics_cols].iloc[grid.best_index_]
print(grid.best_estimator_)
print(final_metrics)
'''

"\nmodel = DecisionTreeClassifier(random_state=42)\nparams = {\n    'model__max_depth': range(2, 21, 2),\n    'model__criterion': ['gini', 'entropy']\n}\n\npreprocessing = ColumnTransformer([\n    ('vectorizer', vectorizer, 'subs'),\n    ('scaler', scaler, levels),\n])\n\npipe = Pipeline([\n    ('preprocessing', preprocessing),\n    ('model', model)\n])\n\ngrid = GridSearchCV(pipe, params, scoring=scoring, refit='f1_micro')\ngrid.fit(X_train, y_train)\n\nmetrics_cols = [f'mean_test_{x}' for x in scoring]\nfinal_metrics = pd.DataFrame(grid.cv_results_)[metrics_cols].iloc[grid.best_index_]\nprint(grid.best_estimator_)\nprint(final_metrics)\n"

In [198]:
model = DecisionTreeClassifier(random_state=42, max_depth=20, criterion='entropy')
y_pred = get_y_pred(model)
print_f1_score(model, y_pred)

DecisionTreeClassifier(criterion='entropy', max_depth=20, random_state=42):
	f1_micro: 0.6565656565656566
	f1_macro: 0.6514898753585331


### 3. 3. RandomForestClassifier

In [341]:
model = RandomForestClassifier(random_state=42, n_estimators=350, max_depth=20)
y_pred = get_y_pred(model)
print_f1_score(model, y_pred)

RandomForestClassifier(max_depth=20, n_estimators=350, random_state=42):
	f1_micro: 0.6768
	f1_macro: 0.5737


### 3. 4. LogisticRegression

In [324]:
model = LogisticRegression(random_state=42, max_iter=300, C=35)
y_pred = get_y_pred(model)
print_f1_score(model, y_pred)

LogisticRegression(C=35, max_iter=300, random_state=42):
	f1_micro: 0.7677
	f1_macro: 0.7177


### 3. 5. RidgeClassifier

In [322]:
model = RidgeClassifier(random_state=42, alpha=0.1)
y_pred = get_y_pred(model)
print_f1_score(model, y_pred)

RidgeClassifier(alpha=0.1, random_state=42):
	f1_micro: 0.7879
	f1_macro: 0.7646


### 3. 6. SGDClassifier

In [323]:
model = SGDClassifier(random_state=42, loss='huber', alpha=0.0001, penalty='elasticnet', l1_ratio=0.01)
y_pred = get_y_pred(model)
print_f1_score(model, y_pred)

SGDClassifier(l1_ratio=0.01, loss='huber', penalty='elasticnet',
              random_state=42):
	f1_micro: 0.7879
	f1_macro: 0.7754


### 3. 7. LinearSVC

In [321]:
model = LinearSVC(random_state=42, C=3)
y_pred = get_y_pred(model)
print_f1_score(model, y_pred)

LinearSVC(C=3, random_state=42):
	f1_micro: 0.7879
	f1_macro: 0.7646


<a name='analysis'></a>
## 4. Анализ результатов